In [1]:
### Load the required libraries

import warnings
warnings.filterwarnings('ignore')

import sqlite3

import numpy as np
import pandas as pd

import requests
import re
from bs4 import BeautifulSoup

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from tqdm import tqdm, tqdm_notebook
from sentence_transformers import SentenceTransformer, util

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
### To read the useful csv file into DataFrame

df = pd.read_csv('search_engine.csv', nrows=10000)

In [3]:
df.head()

,Unnamed: 0,num,name,file_content,chunk_content
0,17262,9251120,maybe.this.time.(2014).eng.1cd,Watch any video online with OpenSUBTITLES Free...,['Watch any video online with OpenSUBTITLES Fr...
1,7294,9211589,down.the.shore.s01.e10.and.justice.for.all.(19...,Oh I know that its getting late but I dont wan...,['Oh I know that its getting late but I dont w...
2,47707,9380845,uncontrollably.fond.s01.e07.heartache.(2016).e...,Timing and Subtitles by The Uncontrollable Lov...,['Timing and Subtitles by The Uncontrollable L...
3,29914,9301436,screen.two.s13.e04.the.precious.blood.(1996).e...,ethereal music apiOpenSubtitlesorg is deprecat...,['ethereal music apiOpenSubtitlesorg is deprec...
4,54266,9408707,battlebots.(2015).eng.1cd,Chris Oh no not the Minibots yelling Oh You le...,['Chris Oh no not the Minibots yelling Oh You ...


In [4]:
df.iloc[0,4]

"['Watch any video online with OpenSUBTITLES Free Browser extension osdblinkext It couldve been just another summer But as I set foot on the sand that summer suddenly felt different Like it was going to be the summer that would change my life The summer of freedom The summer of endless possibilities The summer of Ooh aah Ooh oh Oh oh ooh That was the summer of you and me Youre quite the dancer Why did you stop Come on Keep dancing Whatever Im kidding Dont get mad Huh What Hey Im just going to get m', 'g Dont get mad Huh What Hey Im just going to get my towel What Stop that You thought I was gonna kiss you No Excuse me I wanna kiss you but not just yet What do you mean not yet Only when youre my girl What do you mean your girl My girlfriend Miss As if You wish And dont call me miss Dont pretend to be a gentleman when youre clearly not So what should I call you Rude Snob Bitch And you Douche Handsome Conceited Just like you Huh Jerk Exactly your type Leave me alone Steph Aha Steph Ill ju

In [5]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [7]:
df['doc_vector_pretrained_bert'] = df.chunk_content.apply(model.encode)

In [8]:
df.head()

,Unnamed: 0,num,name,file_content,chunk_content,doc_vector_pretrained_bert
0,17262,9251120,maybe.this.time.(2014).eng.1cd,Watch any video online with OpenSUBTITLES Free...,['Watch any video online with OpenSUBTITLES Fr...,"[-0.102525264, -0.11649302, 0.02563645, -0.066..."
1,7294,9211589,down.the.shore.s01.e10.and.justice.for.all.(19...,Oh I know that its getting late but I dont wan...,['Oh I know that its getting late but I dont w...,"[-0.023607792, -0.06180392, -0.014973673, -0.0..."
2,47707,9380845,uncontrollably.fond.s01.e07.heartache.(2016).e...,Timing and Subtitles by The Uncontrollable Lov...,['Timing and Subtitles by The Uncontrollable L...,"[-0.041544832, -0.0021895964, 0.022084285, -0...."
3,29914,9301436,screen.two.s13.e04.the.precious.blood.(1996).e...,ethereal music apiOpenSubtitlesorg is deprecat...,['ethereal music apiOpenSubtitlesorg is deprec...,"[-0.019292817, 0.012044347, -0.0034768302, -0...."
4,54266,9408707,battlebots.(2015).eng.1cd,Chris Oh no not the Minibots yelling Oh You le...,['Chris Oh no not the Minibots yelling Oh You ...,"[-0.09477355, -0.004724413, -0.0082431305, -0...."


In [9]:
df['doc_vector_pretrained_bert'][0].shape

(384,)

In [10]:
len(df['doc_vector_pretrained_bert'][0])

384

In [11]:
df.to_csv('se.csv')

In [12]:
df.head()

,Unnamed: 0,num,name,file_content,chunk_content,doc_vector_pretrained_bert
0,17262,9251120,maybe.this.time.(2014).eng.1cd,Watch any video online with OpenSUBTITLES Free...,['Watch any video online with OpenSUBTITLES Fr...,"[-0.102525264, -0.11649302, 0.02563645, -0.066..."
1,7294,9211589,down.the.shore.s01.e10.and.justice.for.all.(19...,Oh I know that its getting late but I dont wan...,['Oh I know that its getting late but I dont w...,"[-0.023607792, -0.06180392, -0.014973673, -0.0..."
2,47707,9380845,uncontrollably.fond.s01.e07.heartache.(2016).e...,Timing and Subtitles by The Uncontrollable Lov...,['Timing and Subtitles by The Uncontrollable L...,"[-0.041544832, -0.0021895964, 0.022084285, -0...."
3,29914,9301436,screen.two.s13.e04.the.precious.blood.(1996).e...,ethereal music apiOpenSubtitlesorg is deprecat...,['ethereal music apiOpenSubtitlesorg is deprec...,"[-0.019292817, 0.012044347, -0.0034768302, -0...."
4,54266,9408707,battlebots.(2015).eng.1cd,Chris Oh no not the Minibots yelling Oh You le...,['Chris Oh no not the Minibots yelling Oh You ...,"[-0.09477355, -0.004724413, -0.0082431305, -0...."


In [13]:
df.drop(df.columns[0],axis=1, inplace=True)

In [14]:
df.head()

,num,name,file_content,chunk_content,doc_vector_pretrained_bert
0,9251120,maybe.this.time.(2014).eng.1cd,Watch any video online with OpenSUBTITLES Free...,['Watch any video online with OpenSUBTITLES Fr...,"[-0.102525264, -0.11649302, 0.02563645, -0.066..."
1,9211589,down.the.shore.s01.e10.and.justice.for.all.(19...,Oh I know that its getting late but I dont wan...,['Oh I know that its getting late but I dont w...,"[-0.023607792, -0.06180392, -0.014973673, -0.0..."
2,9380845,uncontrollably.fond.s01.e07.heartache.(2016).e...,Timing and Subtitles by The Uncontrollable Lov...,['Timing and Subtitles by The Uncontrollable L...,"[-0.041544832, -0.0021895964, 0.022084285, -0...."
3,9301436,screen.two.s13.e04.the.precious.blood.(1996).e...,ethereal music apiOpenSubtitlesorg is deprecat...,['ethereal music apiOpenSubtitlesorg is deprec...,"[-0.019292817, 0.012044347, -0.0034768302, -0...."
4,9408707,battlebots.(2015).eng.1cd,Chris Oh no not the Minibots yelling Oh You le...,['Chris Oh no not the Minibots yelling Oh You ...,"[-0.09477355, -0.004724413, -0.0082431305, -0...."


In [16]:
def search(query, data, embeddings, model):
    
    query_embedding = model.encode([query])[0]
    similarities = cosine_similarity([query_embedding], embeddings)
    
    top_n = 10
    top_indices = np.argsort(similarities[0])[-top_n:][::-1]  
    results = [(data['name'][i], similarities[0][i]) for i in top_indices]
    
    return results

In [17]:
embeddings = np.array(df['doc_vector_pretrained_bert'].tolist())

In [18]:
embedding_dict = {}
for i, embedding in enumerate(embeddings):
    embedding_dict[i] = embedding

for i in range(1):
    print(f"Embedding {i}: {embedding_dict[i]}")

Embedding 0: [-1.02525264e-01 -1.16493016e-01  2.56364495e-02 -6.61284700e-02
  7.81166479e-02  4.39900579e-03  6.99874088e-02 -1.28689026e-02
  2.52963919e-02 -3.03600598e-02  2.17511877e-02 -5.78080229e-02
  2.33948473e-02 -4.23618266e-03  7.87564833e-03  5.76398000e-02
  9.56680849e-02  1.92319416e-02 -5.51475585e-02 -8.79280828e-03
  4.36077788e-02 -8.93852301e-03 -2.88903601e-02  2.35581752e-02
 -6.80454597e-02  1.08474223e-02 -1.89094562e-02  6.43841028e-02
 -3.54106501e-02  5.81730679e-02 -5.08178510e-02  1.53503790e-01
 -2.70549878e-02  9.35446878e-04 -5.27724158e-03  2.21493784e-02
 -5.09897135e-02  2.71035247e-02 -4.35737446e-02  1.04851127e-02
 -2.91671101e-02 -4.40238640e-02 -5.69721591e-03  2.39519030e-02
  9.92820039e-03  1.99032482e-02 -7.69885955e-03 -5.53701408e-02
 -3.96464672e-03 -8.13378394e-02 -3.64094749e-02 -1.22366659e-02
 -3.50115858e-02  4.12600599e-02 -6.08579740e-02 -4.47799750e-02
  8.58690962e-02  3.75339910e-02  8.08226541e-02  2.43651625e-02
 -5.34622110

In [20]:
from sklearn.metrics.pairwise import cosine_similarity

query = input("Enter your search query of English movies and series: ")
search_results = search(query, df, embeddings, model)
for result in search_results:
    print("Document:", result[0])
    print("Similarity Score:", result[1])
    print()

Enter your search query of English movies and series: romantic
Document: radio.romance.(1996).eng.1cd
Similarity Score: 0.41177303

Document: les.enfilees.(1980).eng.1cd
Similarity Score: 0.39734554

Document: somebody.s01.e05.episode.1.5.(2022).eng.1cd
Similarity Score: 0.33645165

Document: down.with.love.(2003).eng.1cd
Similarity Score: 0.33522135

Document: ncis.s02.e15.caught.on.tape.(2005).eng.1cd
Similarity Score: 0.32902145

Document: heartbreak.high.s01.e06.angeline.(2022).eng.1cd
Similarity Score: 0.32580668

Document: the.mindy.project.s06.e01.is.that.all.there.is.(2017).eng.1cd
Similarity Score: 0.3249193

Document: harley.quinn.s03.e11.harley.quinn.a.very.problematic.valentines.day.special.(2023).eng.1cd
Similarity Score: 0.32413346

Document: beverly.hills.90210.s04.e13.emily.(1993).eng.1cd
Similarity Score: 0.3184452

Document: irl.in.real.love.(2023).eng.1cd
Similarity Score: 0.31479716



In [21]:
ids = df.index.astype(str).tolist()
documents = df['chunk_content'].tolist()
metadata = df.drop(['file_content','chunk_content','doc_vector_pretrained_bert'], axis = 1).to_dict(orient = 'records')

In [22]:
documents[0]

"['Watch any video online with OpenSUBTITLES Free Browser extension osdblinkext It couldve been just another summer But as I set foot on the sand that summer suddenly felt different Like it was going to be the summer that would change my life The summer of freedom The summer of endless possibilities The summer of Ooh aah Ooh oh Oh oh ooh That was the summer of you and me Youre quite the dancer Why did you stop Come on Keep dancing Whatever Im kidding Dont get mad Huh What Hey Im just going to get m', 'g Dont get mad Huh What Hey Im just going to get my towel What Stop that You thought I was gonna kiss you No Excuse me I wanna kiss you but not just yet What do you mean not yet Only when youre my girl What do you mean your girl My girlfriend Miss As if You wish And dont call me miss Dont pretend to be a gentleman when youre clearly not So what should I call you Rude Snob Bitch And you Douche Handsome Conceited Just like you Huh Jerk Exactly your type Leave me alone Steph Aha Steph Ill ju

In [23]:
embeddings[0]

array([-1.02525264e-01, -1.16493016e-01,  2.56364495e-02, -6.61284700e-02,
        7.81166479e-02,  4.39900579e-03,  6.99874088e-02, -1.28689026e-02,
        2.52963919e-02, -3.03600598e-02,  2.17511877e-02, -5.78080229e-02,
        2.33948473e-02, -4.23618266e-03,  7.87564833e-03,  5.76398000e-02,
        9.56680849e-02,  1.92319416e-02, -5.51475585e-02, -8.79280828e-03,
        4.36077788e-02, -8.93852301e-03, -2.88903601e-02,  2.35581752e-02,
       -6.80454597e-02,  1.08474223e-02, -1.89094562e-02,  6.43841028e-02,
       -3.54106501e-02,  5.81730679e-02, -5.08178510e-02,  1.53503790e-01,
       -2.70549878e-02,  9.35446878e-04, -5.27724158e-03,  2.21493784e-02,
       -5.09897135e-02,  2.71035247e-02, -4.35737446e-02,  1.04851127e-02,
       -2.91671101e-02, -4.40238640e-02, -5.69721591e-03,  2.39519030e-02,
        9.92820039e-03,  1.99032482e-02, -7.69885955e-03, -5.53701408e-02,
       -3.96464672e-03, -8.13378394e-02, -3.64094749e-02, -1.22366659e-02,
       -3.50115858e-02,  

In [24]:
import chromadb
client = chromadb.PersistentClient(path="/Internship")

In [25]:
collection = client.create_collection(name="Search_Engine", metadata={"hnsw:space": "cosine"})

In [26]:
client.heartbeat()

1714061175524892100

In [29]:
embeddings_as_lists = [embedding.tolist() for embedding in embeddings]

In [30]:
for i, embedding in enumerate(embeddings_as_lists):

    # Add the embeddings list to your collection
    collection.add(
            documents=documents[i],
            embeddings=embeddings_as_lists[i],
            ids=ids[i],
            metadatas=metadata[i]
        )

In [31]:
collection.count()

10000

In [36]:
results = collection.query(query_texts=["Where are you working Im currently jobless  Timing and Subtitles by The Uncontrollable  Chris Oh no not the Minibots yelling Oh You"],n_results=5)

C:\Users\91994\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|█████| 79.3M/79.3M [00:31<00:00, 2.61MiB/s]


In [37]:
results

{'ids': [['4', '9264', '232', '3058', '6701']],
 'distances': [[0.4642068147659302,
   0.5082446336746216,
   0.5310921669006348,
   0.5364724397659302,
   0.5513179302215576]],
 'metadatas': [[{'name': 'battlebots.(2015).eng.1cd', 'num': 9408707},
   {'name': 'friends.s06.e20.the.one.with.mac.and.c.h.e.e.s.e.(2000).eng.1cd',
    'num': 9333963},
   {'name': 'the.middle.s02.e22.the.prom.(2011).eng.1cd', 'num': 9425355},
   {'name': 'taskmaster.nz.s03.e04.herbs.spices.(2022).eng.1cd',
    'num': 9185630},
   {'name': 'mystery.science.theater.3000.s04.e17.the.saga.of.the.viking.women.and.their.voyage.to.the.waters.of.the.great.sea.serpent.(1991).eng.1cd',
    'num': 9309554}]],
 'embeddings': None,
 'documents': [["['Chris Oh no not the Minibots yelling Oh You leave those little bots alone Pick on somebody your own size you creep You bully Lisa On nearly every single episode ofBattleBots a Minibot is abused Chris yelling Oh the humanity Lisa Last year alone over Minibots were rescued fro